# Magliette AzIM: ordini, prezzi e consegne

L'**Azienda degli Ingegneri Matematici** ha bisogno del tuo aiuto: uno stagista chiamato F.B. ha eliminato il database degli ordini delle stupende **magliette ufficiali AzIM**! Tutto quello che rimane sono 420 frammenti di email con le richieste dei clienti, spetta a te recuperarle e determinare che ordini accettare!


0. Nella cartella data trovi tutti i file di testo con i frammenti di email, sono tutti diversi! Contengono la **taglia** delle magliette, la **quantità** di magliette ordinate, la **posizione** in coordinate GCS della consegna, il **nome** del cliente e il **prezzo** a cui vorrebbe acquistare il singolo capo


1. [1pt] Registra gli ordini in una **lista di oggetti** "Order"


2. [1pt] Ogni cliente ha richiesto un prezzo diverso, ma il prezzo di vendita deve essere unico. Considerando 6€ come costo di produzione di una maglietta, determina il **prezzo di vendita** a cui si otterrebbe il maggior  profitto totale (se il prezzo richiesto dal cliente è strettamente inferiore al prezzo di vendita, il cliente non acquisterà)


3. [1pt] Considera i **costi di consegna**: viene applicato un costo di 0.10€/km a capo per per ogni ordine sotto le 100 unità, mentre di 0.05€/km per ogni ordine di almeno 100 unità. Trova il nuovo prezzo di vendita che porti al maggior profitto totale e componi la lista degli ordini da eseguire, riordinata dall'ordine che porterebbe al profitto maggiore fino a quella che porterebbe al profitto minore.


4. [FACOLTATIVO per lode] Considera la curvatura terrestre per il calcolo della distanza per le consegne (riscrivi la funzione "Distance")


Utilizza il seguente codice come punto di partenza, riempiendo gli spazi dove manca codice!


Per qualsiasi domanda contattare Jean Paul G. Baroni +393315969286

In [1]:
# 1. Registra gli ordini in una lista di classi Order

GCS_Piazza_Leo = [45.478104, 9.227040] # Coordinate di Piazza Leonardo da Vinci, Milano
production_cost = 6 # Costo fisso di produzione della maglietta
num_orders = 420 # Numero di ordini nella cartella "data"

orders = [] # Lista di oggetti che descrivono gli ordini ricevuti

# Classe che descrive ciascun ordine ricevuto
class Order:
    
    # Costruttore della classe (prende in input il frammento di email salvato nei .txt)
    def __init__(self, email_fragment):
        
        # Separa le righe del frammento e le salva nella lista lines
        lines = email_fragment.split("\n")
        
        # Salva la taglia della maglietta
        self.size = lines[1].split(" ")[-1]
        
        # Salva la quantità di magliette richieste
        self.quantity = float(lines[1].split(" ")[-5])
        
        # Salva la posizione di consegna dell'ordine come due elementi float di una lista
        self.GCS = [float(lines[2].split(" ")[-1].split(",")[0][1:]),float(lines[2].split(" ")[-1].split(",")[1][:-1])]
        
        # Salva il nome del cliente (ultima riga del txt)
        self.customer = lines[-1]
        
        # Salva il prezzo richiesto dal cliente (bid price)
        self.bid_price = float(lines[-3].split(" ")[-1].replace("€","").rstrip())
    
    # Calcola il pagamento che questo cliente effettuerebbe dato un prezzo di vendita
    def Payment(self, ask_price):
        return (self.quantity*ask_price)
    
    # Calcola il profitto (ricavi - costi) per un ordine a questo cliente dato un prezzo di vendita
    def Profit(self, ask_price, cost_per_km = 0):
        profitto=(ask_price-production_cost)*self.quantity
        return profitto-self.Distance()*cost_per_km*self.quantity
            
                
    # Calcola una distanza approssimata in km tra piazza Leonardo da Vinci (punto di partenza delle consegne) e la destinazione
    def Distance(self, starting_GCS = GCS_Piazza_Leo):
        return (((starting_GCS[0] - self.GCS[0])*111)**2+((starting_GCS[1] - self.GCS[1])*78)**2)**0.5
    
    # Stampa i dettagli dell'oggetto
    def Details(self):
        print(self.customer,"ha ordinato",self.quantity,"magliette taglia",self.size,"a "+str(self.bid_price)+"€ all'indirizzo",self.GCS)
    
# Crea gli oggetti Order e li registra nella lista orders
for i in range(num_orders):
    _######## SCRIVI CODICE ########
    # Apri il file ... orders.append( ... Chiudi il file
    file = open(f"{i+1}.txt","r")
    testo = file.read()
    ordine=Order(testo)
    orders.append(ordine)
    file.close()

In [2]:
# 2. [1pt] Ogni cliente ha richiesto un prezzo diverso, ma il prezzo di vendita deve essere unico. Considerando 6€ come costo di produzione di una maglietta, determina il **prezzo di vendita** a cui si otterrebbe il maggior  profitto totale (se il prezzo richiesto dal cliente è strettamente inferiore al prezzo di vendita, il cliente non acquisterà)

# Determina i profitti per tutti i possibili prezzi di vendita, riferendosi ai prezzi richiesti dai clienti 
possible_profits = [ elem.Profit(elem.bid_price) for elem in orders ]
#print(possible_profits)

# Determina il profitto maggiore
max_profit = max(possible_profits)

# Determina il prezzo migliore
best_price = orders[possible_profits.index(max(possible_profits))].bid_price
print("Il miglior prezzo è €"+str(best_price))

# Trova gli ordini da eseguire dato il miglior prezzo
orders_to_go = [ord1 for ord1 in orders if ord1.bid_price>=best_price]
print("Si eseguiranno", len(orders_to_go),"ordini")

Il miglior prezzo è €19.85
Si eseguiranno 11 ordini


In [3]:
# 3. [1pt] Considera i **costi di consegna**: viene applicato un costo di 0.10€/km a capo per per ogni ordine sotto le 100 unità, mentre di 0.05€/km per ogni ordine di almeno 100 unità. Trova il nuovo prezzo di vendita che porti al maggior profitto totale e componi la lista degli ordini da eseguire, riordinata dall'ordine che porterebbe al profitto maggiore fino a quella che porterebbe al profitto minore.

cost_less_100 = 0.10
cost_at_least_100 = 0.05

# Determina i profitti per tutti i possibili prezzi di vendita, riferendosi ai prezzi richiesti dai clienti 
possible_profits = [ elem.Profit(elem.bid_price, cost_less_100) if elem.quantity<100 else elem.Profit(elem.bid_price, cost_at_least_100) for elem in orders]
#_ = [print(pp) for pp in possible_profits]

# Determina il profitto maggiore
max_profit = max(possible_profits)

# Determina il prezzo migliore
best_price = orders[possible_profits.index(max(possible_profits))].bid_price
print("Il miglior prezzo è €"+str(best_price))

# Trova gli ordini da eseguire dato il miglior prezzo
orders_to_go = [ord1 for ord1 in orders if ord1.bid_price>=best_price]
print("Si eseguiranno", len(orders_to_go),"ordini")

######## SCRIVI CODICE ########
# ... assembla la lista degli ordini che devono che si possono soddisfare, dando precedenza a quelli che portano maggiore profitto
######## SCRIVI CODICE ########
possible_profits2 = [ elem.Profit(best_price, cost_less_100) if elem.quantity<100 else elem.Profit(best_price, cost_at_least_100) for elem in orders_to_go]

def find_index(lista_ordini, maxprofitto):
    for i in range (len(lista_ordini)):
        if lista_ordini[i].quantity<100:
            costo=cost_less_100
        else:
            costo=cost_at_least_100
        if lista_ordini[i].Profit(best_price,costo)==maxprofitto:
            return i
        
b=possible_profits2

for i in range(len(possible_profits2)):
    massimo=max(b)
    b.remove(massimo)
    j=find_index(orders_to_go,massimo)
    orders_to_go[i],orders_to_go[j]=orders_to_go[j],orders_to_go[i]


_ = [otg.Details() for otg in orders_to_go]


Il miglior prezzo è €19.62
Si eseguiranno 17 ordini
Davide Baroni Frumento ha ordinato 431.0 magliette taglia S a 19.62€ all'indirizzo [45.755771, 8.645121]
Gianmarco Bressan Company ha ordinato 491.0 magliette taglia XS a 19.79€ all'indirizzo [45.661732, 10.428504]
JP Carrara Servizi Finanziari ha ordinato 486.0 magliette taglia XS a 19.87€ all'indirizzo [46.177794, 8.410633]
MC Zacchei Latte ha ordinato 351.0 magliette taglia XS a 19.88€ all'indirizzo [45.498935, 8.319235]
Filippo Cerisola Formaggi ha ordinato 451.0 magliette taglia L a 19.75€ all'indirizzo [46.430318, 8.170284]
Manuel Carrara Franchising ha ordinato 492.0 magliette taglia XS a 19.85€ all'indirizzo [44.811402, 11.070849]
Alessandro Bressan IB ha ordinato 242.0 magliette taglia XS a 19.99€ all'indirizzo [45.152525, 8.134165]
JP Cerisola Giochi d'Azzardo ha ordinato 156.0 magliette taglia L a 19.86€ all'indirizzo [45.337628, 9.235005]
Filippo Zacchei SRL ha ordinato 353.0 magliette taglia S a 19.91€ all'indirizzo [45.4